In [2]:
import requests # The requests library is an 
import bs4 as bs # BeautifulSoup4 is a Python library 
import pandas as pd
from collections import Counter
import re

In [4]:
df = pd.read_csv('data/airbnb-listings.csv', sep=',', engine='python')
df = pd.DataFrame(df)
df.head()

ID                            Listing Url  Scrape ID Last Scraped  \
0 NaN   https://www.airbnb.com/rooms/2105840        NaN       4/2/17   
1 NaN   https://www.airbnb.com/rooms/5505643        NaN       4/2/17   
2 NaN    https://www.airbnb.com/rooms/746079        NaN       4/2/17   
3 NaN  https://www.airbnb.com/rooms/11680405        NaN       4/2/17   
4 NaN   https://www.airbnb.com/rooms/1304068        NaN       4/2/17   

                                  Name  \
0  Room w/priv. bath Duboce Triangle 1   
1     Modern Mission-Dolores apartment   
2         Castro/Corbett Heights Condo   
3      Sun-filled, Spacious Duboce 1BR   
4      Historic SF Residence on a park   

                                             Summary  \
0  Stay with us in our live/work space and become...   
1  Top floor apartment in a two unit building, bu...   
2  Enjoy this quiet bedroom suite in my luxury Co...   
3  Our airy, spacious 1 BR in the Duboce/Castro a...   
4  Central location, well maintained Victorian, g...   

                                               Space  \
0  Stay with us in our live/work space and become...   
1  Full open floor living room/dining room/kitche...   
2  Enjoy this quiet bedroom suite in my luxury Co...   
3  Both the bedroom and living room have bay wind...   
4  Central location, well maintained Victorian, g...   

                                         Description Experiences Offered  \
0  Stay with us in our live/work space and become...                none   
1  Top floor apartment in a two unit building, bu...                none   
2  Enjoy this quiet bedroom suite in my luxury Co...                none   
3  Our airy, spacious 1 BR in the Duboce/Castro a...                none   
4  Central location, well maintained Victorian, g...                none   

                               Neighborhood Overview    ...     \
0  We think that this is the best neighborhood in...    ...      
1  Fantastic location at the intersection of Miss...    ...      
2  Locals call my neighborhood "Corbett Heights" ...    ...      
3  This is the best situated neighborhood in SF. ...    ...      
4                                                NaN    ...      

  Review Scores Communication Review Scores Location Review Scores Value  \
0                        10.0                   10.0                 9.0   
1                        10.0                   10.0                10.0   
2                        10.0                   10.0                 9.0   
3                        10.0                   10.0                10.0   
4                        10.0                   10.0                10.0   

                                             License Jurisdiction Names  \
0                                                NaN      SAN FRANCISCO   
1                                                NaN      SAN FRANCISCO   
2  SAN FRANCISCO SHORT-TERM RENTAL REGISTRATION N...      SAN FRANCISCO   
3                                                NaN      SAN FRANCISCO   
4                                        STR-0001041      SAN FRANCISCO   

  Cancellation Policy Calculated host listings count  Reviews per Month  \
0              strict                              2               1.16   
1              strict                              1               2.51   
2            flexible                              1               4.30   
3            moderate                              1               0.42   
4            moderate                              1               0.23   

  Geolocation  Features  
0         NaN       NaN  
1         NaN       NaN  
2         NaN       NaN  
3         NaN       NaN  
4         NaN       NaN  

[5 rows x 89 columns]

In [34]:
# example of expired site 
source = requests.get("https://www.airbnb.com/rooms/841083") 
soup = bs.BeautifulSoup(source.content, features='html.parser')
print(soup)
title = soup.find('title')
print(title.text)

<!DOCTYPE html>

<html dir="ltr" lang="en" xmlns:fb="http://ogp.me/ns/fb#">
<head>
<meta charset="utf-8"/>
<link href="https://a0.muscache.com/airbnb/static/packages/dls/common_o2.1_cereal-4babec524f2f65616d1bec719d57bb38.css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://a0.muscache.com/airbnb/static/packages/common-7d0f582f30cdc661f9c474f22e4da235.css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://a0.muscache.com/airbnb/static/packages/map_search-13a1ad83f0e873b8e939e171a0e23964.css" media="screen" rel="stylesheet" type="text/css"/><link href="https://a0.muscache.com/airbnb/static/packages/p3-cf46e612e26c5bdfae9d01a9c28e166c.css" media="screen" rel="stylesheet" type="text/css"/>
<script>

      var sherlock_firstbyte = window.performance && window.performance.timing
        ? window.performance.timing.responseStart
        : Number(new Date());
      !function(){if(window.PerformanceLongTaskTiming){var g=window.__tti={e:[]};
      g.o=new Per

From data set, we found some url expired in Listing Url column so we can't get price information by web scraping.
Let's keep non-expire url data set only.

In [36]:
# check the website is expired so we can't get price information ( expire = 1 , otherwise = 0)
expire = list()
for i in df['Listing Url']:
    source = requests.get(i) 
    soup = bs.BeautifulSoup(source.content, features='html.parser')
    title = soup.find('title')
    if title.text == ("Vacation Rentals, Homes, Experiences & Places - Airbnb"):
        expire.append(1)
    else:
        expire.append(0)
        

In [40]:
# total nrow of data set
len(expire)

8680

In [41]:
# total number of non-expired url
expire.count(0)

2839

In [42]:
# add column "expire" in df
df['expire']=expire

In [44]:
df.head(5)

ID                            Listing Url  Scrape ID Last Scraped  \
0 NaN   https://www.airbnb.com/rooms/2105840        NaN       4/2/17   
1 NaN   https://www.airbnb.com/rooms/5505643        NaN       4/2/17   
2 NaN    https://www.airbnb.com/rooms/746079        NaN       4/2/17   
3 NaN  https://www.airbnb.com/rooms/11680405        NaN       4/2/17   
4 NaN   https://www.airbnb.com/rooms/1304068        NaN       4/2/17   

                                  Name  \
0  Room w/priv. bath Duboce Triangle 1   
1     Modern Mission-Dolores apartment   
2         Castro/Corbett Heights Condo   
3      Sun-filled, Spacious Duboce 1BR   
4      Historic SF Residence on a park   

                                             Summary  \
0  Stay with us in our live/work space and become...   
1  Top floor apartment in a two unit building, bu...   
2  Enjoy this quiet bedroom suite in my luxury Co...   
3  Our airy, spacious 1 BR in the Duboce/Castro a...   
4  Central location, well maintained Victorian, g...   

                                               Space  \
0  Stay with us in our live/work space and become...   
1  Full open floor living room/dining room/kitche...   
2  Enjoy this quiet bedroom suite in my luxury Co...   
3  Both the bedroom and living room have bay wind...   
4  Central location, well maintained Victorian, g...   

                                         Description Experiences Offered  \
0  Stay with us in our live/work space and become...                none   
1  Top floor apartment in a two unit building, bu...                none   
2  Enjoy this quiet bedroom suite in my luxury Co...                none   
3  Our airy, spacious 1 BR in the Duboce/Castro a...                none   
4  Central location, well maintained Victorian, g...                none   

                               Neighborhood Overview   ...    \
0  We think that this is the best neighborhood in...   ...     
1  Fantastic location at the intersection of Miss...   ...     
2  Locals call my neighborhood "Corbett Heights" ...   ...     
3  This is the best situated neighborhood in SF. ...   ...     
4                                                NaN   ...     

  Review Scores Location Review Scores Value  \
0                   10.0                 9.0   
1                   10.0                10.0   
2                   10.0                 9.0   
3                   10.0                10.0   
4                   10.0                10.0   

                                             License Jurisdiction Names  \
0                                                NaN      SAN FRANCISCO   
1                                                NaN      SAN FRANCISCO   
2  SAN FRANCISCO SHORT-TERM RENTAL REGISTRATION N...      SAN FRANCISCO   
3                                                NaN      SAN FRANCISCO   
4                                        STR-0001041      SAN FRANCISCO   

  Cancellation Policy Calculated host listings count Reviews per Month  \
0              strict                              2              1.16   
1              strict                              1              2.51   
2            flexible                              1              4.30   
3            moderate                              1              0.42   
4            moderate                              1              0.23   

   Geolocation Features  expire  
0          NaN      NaN       0  
1          NaN      NaN       0  
2          NaN      NaN       0  
3          NaN      NaN       1  
4          NaN      NaN       0  

[5 rows x 90 columns]

In [50]:
# drop expire-website(expire = 1) and keep non-expire one (expire = 0)
df = df[df.expire != 1]

In [52]:
df.to_csv('data/clean-airbnb.csv', sep='\t')

In [2]:
import requests # The requests library is an 
import bs4 as bs # BeautifulSoup4 is a Python library 
import pandas as pd
from collections import Counter
import re
df = pd.read_csv('data/airbnb-listings.csv', sep=',', engine='python')
df = pd.DataFrame(df)
df.head()

ID                            Listing Url  Scrape ID Last Scraped  \
0 NaN   https://www.airbnb.com/rooms/2105840        NaN       4/2/17   
1 NaN   https://www.airbnb.com/rooms/5505643        NaN       4/2/17   
2 NaN    https://www.airbnb.com/rooms/746079        NaN       4/2/17   
3 NaN  https://www.airbnb.com/rooms/11680405        NaN       4/2/17   
4 NaN   https://www.airbnb.com/rooms/1304068        NaN       4/2/17   

                                  Name  \
0  Room w/priv. bath Duboce Triangle 1   
1     Modern Mission-Dolores apartment   
2         Castro/Corbett Heights Condo   
3      Sun-filled, Spacious Duboce 1BR   
4      Historic SF Residence on a park   

                                             Summary  \
0  Stay with us in our live/work space and become...   
1  Top floor apartment in a two unit building, bu...   
2  Enjoy this quiet bedroom suite in my luxury Co...   
3  Our airy, spacious 1 BR in the Duboce/Castro a...   
4  Central location, well maintained Victorian, g...   

                                               Space  \
0  Stay with us in our live/work space and become...   
1  Full open floor living room/dining room/kitche...   
2  Enjoy this quiet bedroom suite in my luxury Co...   
3  Both the bedroom and living room have bay wind...   
4  Central location, well maintained Victorian, g...   

                                         Description Experiences Offered  \
0  Stay with us in our live/work space and become...                none   
1  Top floor apartment in a two unit building, bu...                none   
2  Enjoy this quiet bedroom suite in my luxury Co...                none   
3  Our airy, spacious 1 BR in the Duboce/Castro a...                none   
4  Central location, well maintained Victorian, g...                none   

                               Neighborhood Overview    ...     \
0  We think that this is the best neighborhood in...    ...      
1  Fantastic location at the intersection of Miss...    ...      
2  Locals call my neighborhood "Corbett Heights" ...    ...      
3  This is the best situated neighborhood in SF. ...    ...      
4                                                NaN    ...      

  Review Scores Communication Review Scores Location Review Scores Value  \
0                        10.0                   10.0                 9.0   
1                        10.0                   10.0                10.0   
2                        10.0                   10.0                 9.0   
3                        10.0                   10.0                10.0   
4                        10.0                   10.0                10.0   

                                             License Jurisdiction Names  \
0                                                NaN      SAN FRANCISCO   
1                                                NaN      SAN FRANCISCO   
2  SAN FRANCISCO SHORT-TERM RENTAL REGISTRATION N...      SAN FRANCISCO   
3                                                NaN      SAN FRANCISCO   
4                                        STR-0001041      SAN FRANCISCO   

  Cancellation Policy Calculated host listings count  Reviews per Month  \
0              strict                              2               1.16   
1              strict                              1               2.51   
2            flexible                              1               4.30   
3            moderate                              1               0.42   
4            moderate                              1               0.23   

  Geolocation  Features  
0         NaN       NaN  
1         NaN       NaN  
2         NaN       NaN  
3         NaN       NaN  
4         NaN       NaN  

[5 rows x 89 columns]